In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import linregress, sem
from statistics import mean, median
from math import isfinite
from sklearn.metrics import mean_absolute_error

# ignore 'nan' values rather than throw a Runtime warning
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
from sklearn.utils import resample

# bootstrap confidence intervals
# repeatedly resample the data into random subsets
# .. take the mean and medians of the subset
# .. you can then use numpy.percentile() to get CI
def boot_sample(list, iterations=1000, fraction=0.75):
    size = int( len(list) * fraction)
    means = []
    medians = []
    for i in range(iterations):
        subset = resample(list, n_samples=size)
        means.append(mean(subset))
        medians.append(median(subset))
    return means, medians

In [3]:
df = pd.read_csv('data-nodispersion.csv')
df.columns

Index(['name', 'geom', 'natoms', 'dlpno', 'wb97', 'pbe', 'pbeSVP', 'b3lypTZ',
       'b3lypSVP'],
      dtype='object')

In [4]:
df.dtypes # make sure the numbers are all float64

name         object
geom         object
natoms        int64
dlpno       float64
wb97        float64
pbe         float64
pbeSVP      float64
b3lypTZ     float64
b3lypSVP    float64
dtype: object

In [5]:
# this list allows you to pick only a subset for analysis
methods = ['dlpno', 'wb97', 'pbe', 'pbeSVP', 'b3lypTZ', 'b3lypSVP']

In [6]:
names = np.unique(df['name']) # all the molecule names
print(names)

['astex_1g9v' 'astex_1gkc' 'astex_1gm8' 'astex_1hnn' 'astex_1hp0'
 'astex_1hq2' 'astex_1hvy' 'astex_1hwi' 'astex_1ia1' 'astex_1ig3'
 'astex_1j3j' 'astex_1jd0' 'astex_1jje' 'astex_1jla' 'astex_1k3u'
 'astex_1ke5' 'astex_1kzk' 'astex_1l2s' 'astex_1l7f' 'astex_1lpz'
 'astex_1lrh' 'astex_1m2z' 'astex_1meh' 'astex_1mmv' 'astex_1mzc'
 'astex_1n1m' 'astex_1n2j' 'astex_1n2v' 'astex_1n46' 'astex_1nav'
 'astex_1of1' 'astex_1of6' 'astex_1opk' 'astex_1oq5' 'astex_1owe'
 'astex_1oyt' 'astex_1p2y' 'astex_1p62' 'astex_1pmn' 'astex_1q1g'
 'astex_1q41' 'astex_1q4g' 'astex_1r1h' 'astex_1r55' 'astex_1r58'
 'astex_1r9o' 'astex_1s3v' 'astex_1sg0' 'astex_1sj0' 'astex_1sq5'
 'astex_1t40' 'astex_1t46' 'astex_1t9b' 'astex_1tow' 'astex_1tt1'
 'astex_1tz8' 'astex_1u1c' 'astex_1u4d' 'astex_1uml' 'astex_1unl'
 'astex_1uou' 'astex_1v0p' 'astex_1v48' 'astex_1v4s' 'astex_1vcj'
 'astex_1w1p' 'astex_1w2g' 'astex_1xm6' 'astex_1xoq' 'astex_1xoz'
 'astex_1y6b' 'astex_1ygc' 'astex_1yqy' 'astex_1yvf' 'astex_1ywr'
 'astex_1z

In [7]:
# we're going to loop through method X and method Y
# SINCE THIS IS FOR THE NON-DISPERSION-CORRECTED we tweak this
with open('summary/data-nondispersion.csv', 'w') as out:
    print("method1, method2, mean mare, median mare, sem mare, std mare, low ci mare, high ci mare, mean rsq, med rsq, sem rsq, std rsq, low ci rsq, high ci rsq, mean spearman, med spearman, sem spearman, std spearman, low ci spearman, high ci spearman", sep=',', file=out)
    for i in range(1):
        for j in range(1, len(methods)):
            mare = []
            r2 = []
            sp = []

            # now we loop through each of the unique molecules
            with open('stats/%s_%s-nod_stats.csv' % (methods[i], methods[j]), 'w') as o:
                print("name, len, mare, rsq, spearman, slope, intercept", sep=',', file=o)
                for name in names:
                    x = df[df['name'] == name][methods[i]]
                    y = df[df['name'] == name][methods[j]]
                    d = pd.DataFrame({'x': x, 'y': y})
                    d = d.dropna()
                    ymax = d['y'].max()
                    d['y_rel'] = d['y'] - ymax

                    xmax = d['x'].max()
                    d['x_rel'] = d['x'] - xmax

                    if len(d) < 3:
                        pass
                    else:
                        mean_abs_rel_energy = mean_absolute_error(d['x_rel'], d['y_rel'])
                        mare.append(mean_abs_rel_energy)
                        d.dropna(subset=['y'], inplace = True)
                        mask = ~np.isnan(x) & ~np.isnan(y)
                        if len(y[mask]) == 0:
                            continue # this molecule has no values (e.g., no DLPNO energies at all)

                        spearman = d.corr(method='spearman').values[0, 1]
                        slope, intercept, r_value, p_value, std_err = linregress(x[mask], y[mask])
                        rsq = r_value**2
                        print(name, len(x), mean_abs_rel_energy, rsq, spearman, slope, intercept, sep=',', file=o)

                        if isfinite(rsq):
                            r2.append(rsq)
                        if isfinite(spearman):
                            sp.append(spearman)
                            
            # okay, now summarize
            if len(r2) > 0 and len(sp) > 0:

                # bootstrap the mare, r2, and spearman values to get confidence intervals
                means, medians = boot_sample(mare)
                med = median(mare)
                low_cim = med - np.percentile(medians, 2.5)
                high_cim = np.percentile(medians, 97.5) - med
                
                means, medians = boot_sample(r2)
                med = median(r2)
                low_cir = med - np.percentile(medians, 2.5)
                high_cir = np.percentile(medians, 97.5) - med
                
                means, medians = boot_sample(sp)
                med = median(sp)
                low_cis = med - np.percentile(medians, 2.5)
                high_cis = np.percentile(medians, 97.5) - med

                print(methods[i], methods[j], mean(mare), median(mare), sem(mare), np.std(mare), low_cim, high_cim, mean(r2), median(r2), sem(r2), np.std(r2), low_cir, high_cir, abs(mean(sp)), abs(median(sp)), abs(sem(sp)), abs(np.std(sp)), low_cis, high_cis, sep=',', file=out)
                print(methods[i], methods[j], mean(mare), median(mare), sem(mare), np.std(mare), low_cim, high_cim, mean(r2), median(r2), sem(r2), np.std(r2), low_cir, high_cir, abs(mean(sp)), abs(median(sp)), abs(sem(sp)), abs(np.std(sp)), low_cis, high_cis)

dlpno wb97 0.34855818678946116 0.18627492501418602 0.016895926167480903 0.4438196723099444 0.014380384584410433 0.02192189787151619 0.7800098529667726 0.8811278829346552 0.009649985257833721 0.25348437561065107 0.018876158473067006 0.020190185164942864 0.8011264182031186 0.8909090909090909 0.008903838548264154 0.2338847049649736 0.018181818181818077 0.0121212121212122
dlpno pbe 0.6414129104402576 0.2994700894327025 0.033007719183171656 0.868297524911353 0.0401005160262457 0.05105376673812312 0.6417041375060097 0.7462286387249016 0.01197610370842821 0.31504210122496223 0.03971938305010336 0.04217817584999228 0.6582597777402972 0.806060606060606 0.014156572835497755 0.37240128850093024 0.036363636363636376 0.024242424242424288
dlpno pbeSVP 0.6262079098814398 0.3315474766429361 0.030005600697796734 0.7893241176342257 0.04632527153328969 0.04184883275120227 0.6445805143520315 0.7509624643771048 0.011735695470485077 0.3087179478711386 0.049203737522668045 0.03655334916532649 0.6740028077708